# Libaries and model import





In [ ]:
from google.colab import files
uploaded = files.upload()

Saving clothing_recognition_deep_cnn_gradio_modelbuilder.h5 to clothing_recognition_deep_cnn_gradio_modelbuilder (2).h5


In [ ]:
import os
print("Uploaded:", os.listdir())

Uploaded: ['.config', 'clothing_recognition_deep_cnn_gradio_modelbuilder (1).h5', '.gradio', 'clothing_recognition_deep_cnn_gradio_modelbuilder.h5', 'clothing_recognition_deep_cnn_gradio_modelbuilder (2).h5', 'sample_data']


In [ ]:
!pip install gradio
!pip install opencv-python-headless

#Importing necessary libraries

In [ ]:
import gradio as gr                   # Gradio for building the web app
import numpy as np                   # Numpy for numerical operations
import tensorflow as tf              # TensorFlow for loading model and predicting
import matplotlib.pyplot as plt      # Matplotlib to generate Grad-CAM visualisations
import cv2                           # OpenCV for image processing
from PIL import Image                # PIL to ensure image compatibility with Gradio
import io

# Load pre-trained CNN model (from Stage 3)

In [ ]:
# Load the model as-is
model = tf.keras.models.load_model("clothing_recognition_deep_cnn_gradio_modelbuilder.h5")

# Call it once to build and define .output/.input
_ = model(tf.zeros((1, 28, 28, 1)))

# Now this will work without errors
print(model.input)
print(model.output)
model.summary()

<KerasTensor shape=(None, 28, 28, 1), dtype=float32, sparse=False, name=input_layer>
<KerasTensor shape=(None, 10), dtype=float32, sparse=False, name=keras_tensor_159>


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 242,444 (947.05 KB)

 Trainable params: 241,994 (945.29 KB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 2 (12.00 B)

# Define class names as per Fashion MNIST standard

In [ ]:
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]


# Preprocess image for model

In [ ]:
def preprocess_image(img):
    img = img.convert("L").resize((28, 28))          # Convert to grayscale and resize
    arr = np.array(img) / 255.0                      # Normalize pixel values to 0–1
    arr = np.expand_dims(arr, axis=-1)               # Add channel dimension: (28, 28, 1)
    input_tensor = np.expand_dims(arr, axis=0)       # Add batch dimension: (1, 28, 28, 1)
    return input_tensor, arr

# Generate Grad-CAM heatmap

In [ ]:
def generate_gradcam(model, image_tensor, last_conv_layer_name="conv2d_2"):
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image_tensor)
        pred_index = tf.argmax(predictions[0])
        class_output = predictions[:, pred_index]
    grads = tape.gradient(class_output, conv_outputs)[0]               # Gradient w.r.t. conv layer
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))                  # Mean of gradients
    conv_outputs = conv_outputs[0]                                     # Remove batch dimension
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]             # Weighted sum
    heatmap = tf.squeeze(heatmap)                                      # Remove last dimension
    heatmap = np.maximum(heatmap, 0)                                   # ReLU activation (only positives)
    heatmap /= tf.reduce_max(heatmap + 1e-8)                           # Normalize heatmap
    return heatmap.numpy()


# Overlay Grad-CAM on original image

In [ ]:
def overlay_heatmap(heatmap, original_image):
    heatmap_resized = cv2.resize(heatmap, (224, 224))                                      # Resize heatmap
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET) # Apply color map
    original_resized = cv2.resize((original_image * 255).astype(np.uint8), (224, 224))     # Resize original image
    original_rgb = cv2.cvtColor(original_resized, cv2.COLOR_GRAY2RGB)                      # Convert grayscale to RGB
    overlay = cv2.addWeighted(original_rgb, 0.6, heatmap_colored, 0.4, 0)                   # Blend overlay
    return overlay


# Gradio Prediction Function

In [ ]:
def predict_and_visualize(uploaded_img):
    input_tensor, processed_image = preprocess_image(uploaded_img)            # Preprocess uploaded image
    prediction = model.predict(input_tensor, verbose=0)[0]                    # Get predictions
    top3_indices = prediction.argsort()[-3:][::-1]                            # Get top-3 class indices
    top3_preds = [(class_names[i], float(prediction[i])) for i in top3_indices]  # Label + confidence

    heatmap = generate_gradcam(model, input_tensor)                           # Grad-CAM computation
    overlay_img = overlay_heatmap(heatmap, processed_image)                   # Overlay heatmap

    prediction_text = "\n".join([                                             # Format top-3 results for textbox
        f"{i+1}. {label} ({conf:.2f})"
        for i, (label, conf) in enumerate(top3_preds)
    ])

    original_img = Image.fromarray((processed_image.squeeze() * 255).astype(np.uint8)).resize((224, 224))
    heatmap_img = Image.fromarray(cv2.applyColorMap(np.uint8(cv2.resize(heatmap, (224, 224)) * 255), cv2.COLORMAP_JET))

    return original_img, heatmap_img, Image.fromarray(overlay_img), prediction_text


# Create Gradio Interface

In [ ]:
with gr.Blocks(title="Fashion Classifier + Grad-CAM Visualisation – Ogunbayo Alfred") as demo:
    gr.Markdown("""
    # Fashion Classifier + Grad-CAM Visualisation Demo
    # Built by Ogunbayo Alfred – Personal AI Project
    Upload a clear image of a clothing item on a neutral background.
    The model predicts the class and highlights regions it used with Grad-CAM.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(type="pil", label="Upload Clothing Image")        # Image uploader input
            predict_btn = gr.Button("Predict & Visualize")                           # Submit button
        with gr.Column(scale=2):
            original_output = gr.Image(label="Original")              # Show input image
            heatmap_output = gr.Image(label="Grad-CAM Heatmap")    # Grad-CAM visualization
            overlay_output = gr.Image(label="Overlay")             # Heatmap on image
            prediction_output = gr.Textbox(label="Top-3 Predictions")                # Prediction textbox

    predict_btn.click(
        fn=predict_and_visualize,
        inputs=image_input,
        outputs=[original_output, heatmap_output, overlay_output, prediction_output]  # Hook all outputs
    )

    gr.Markdown("""
    ---
    ### Project Details:
    • Input: Any image of a clothing item (JPG/PNG)
    • Output: Class label with confidence, Grad-CAM explanation
    • Author: Ogunbayo Alfred | [GitHub](https://github.com/freddylags) | Personal AI Project
    """)


# Launch the App (public Gradio link

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://97c5b2401f80c13dd7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://97c5b2401f80c13dd7.gradio.live


# uploading model to hugging face

In [ ]:
!pip install -q huggingface_hub
from huggingface_hub import login

login()

In [ ]:
from huggingface_hub import create_repo

# Create the repo (switch from 'name' to 'repo_id')
create_repo(
    repo_id="alfred-ogunbayo/fashion-mnist-gradio-demo-model",  # username/repo-name
    repo_type="model",
    exist_ok=True,
    private=False  # Change to True if you want the repo private
)

RepoUrl('https://huggingface.co/alfred-ogunbayo/fashion-mnist-gradio-demo-model', endpoint='https://huggingface.co', repo_type='model', repo_id='alfred-ogunbayo/fashion-mnist-gradio-demo-model')

In [ ]:
from huggingface_hub import upload_file

# Upload model file into the created repo
upload_file(
    path_or_fileobj="clothing_recognition_deep_cnn_gradio_modelbuilder.h5",
    path_in_repo="clothing_recognition_deep_cnn_gradio_modelbuilder.h5",
    repo_id="alfred-ogunbayo/fashion-mnist-gradio-demo-model",
    repo_type="model"
)

clothing_recognition_deep_cnn_gradio_modelbuilder.h5:   0%|          | 0.00/2.98M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alfred-ogunbayo/fashion-mnist-gradio-demo-model/commit/c6334c35bda17cc7d16419fcd6d50d8a3378609a', commit_message='Upload clothing_recognition_deep_cnn_gradio_modelbuilder.h5 with huggingface_hub', commit_description='', oid='c6334c35bda17cc7d16419fcd6d50d8a3378609a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alfred-ogunbayo/fashion-mnist-gradio-demo-model', endpoint='https://huggingface.co', repo_type='model', repo_id='alfred-ogunbayo/fashion-mnist-gradio-demo-model'), pr_revision=None, pr_num=None)

In [ ]:
|